# Introduction to DEA Coastlines <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** [DEA Coastlines](https://cmi.ga.gov.au/data-products/dea/581/dea-coastlines)

## Background
Australia has a highly dynamic coastline of over 30,000 km, with over 85% of its population living within 50 km of the coast. 
This coastline is subject to a wide range of pressures, including extreme weather and climate, sea level rise and human development. 
Understanding how the coastline responds to these pressures is crucial to managing this region, from social, environmental and economic perspectives. 

### What this product offers
[Digital Earth Australia Coastlines](https://maps.dea.ga.gov.au/#share=s-DEACoastlines&playStory=1) is a continental dataset that includes annual shorelines and rates of coastal change along the entire Australian coastline from 1988 to the present. 

The product combines satellite data from Geoscience Australia's Digital Earth Australia program with tidal modelling to map the typical location of the coastline at mean sea level for each year. 
The product enables trends of coastal erosion and growth to be examined annually at both a local and continental scale, and for patterns of coastal change to be mapped historically and updated regularly as data continues to be acquired. 
This allows current rates of coastal change to be compared with that observed in previous years or decades. 

The ability to map shoreline positions for each year provides valuable insights into whether changes to our coastline are the result of particular events or actions, or a process of more gradual change over time. 
This information can enable scientists, managers and policy makers to assess impacts from the range of drivers impacting our coastlines and potentially assist planning and forecasting for future scenarios. 

### Applications
* Monitoring and mapping rates of coastal erosion along the Australian coastline 
* Prioritise and evaluate the impacts of local and regional coastal management based on historical coastline change 
* Modelling how coastlines respond to drivers of change, including extreme weather events, sea level rise or human development 
* Supporting geomorphological studies of how and why coastlines have changed across time 

### Publications

* Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. *Remote Sensing of Environment*, 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734
* Bishop-Taylor, R., Sagar, S., Lymburner, L., Alam, I., & Sixsmith, J. (2019). Sub-pixel waterline extraction: Characterising accuracy and sensitivity to indices and spectra. *Remote Sensing*, 11(24), 2984. Available: https://www.mdpi.com/2072-4292/11/24/2984

> **Note:** For more technical information about the DEA Coastlines product, visit the official Geoscience Australia [DEA Coastlines product description](https://cmi.ga.gov.au/data-products/dea/581/dea-coastlines).

## Description

This notebook will demonstrate how to load data from the [Digital Earth Australia Coastlines](https://cmi.ga.gov.au/data-products/dea/581/dea-coastlines) product using the Digital Earth Australia datacube. 
Topics covered include:

1. Loading DEA Coastlines annual shoreline data using the `get_coastlines` function.
1. Loading DEA Coastlines rates of change statistics data using the `get_coastlines` function.
2. Extracting shoreline data for one or multiple coastal transects using an interactive DEA Coastlines application.

***


## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.coastal import get_coastlines
from dea_tools.app import deacoastlines


## Loading DEA Coastlines data 

### Annual shorelines

DEA Coastlines data can be loaded directly in a Python script or Jupyter Notebook using the DEA Coastlines Web Feature Service (WFS) and `geopandas`:


In [2]:
# Specify bounding box
ymax, xmin = -33.65, 115.28
ymin, xmax = -33.66, 115.30

# Load DEA Coastlines annual shorelines data 
deacl_shorelines_gdf = get_coastlines(bbox=(xmin, ymin, xmax, ymax), 
                                      layer='shorelines')

# Print example output
deacl_shorelines_gdf.head()

,id,year,certainty,maturity,geometry
2,coastlines.676,1990,good,final,"LINESTRING (-1535468.162 -3772746.024, -153547..."
3,coastlines.677,1991,good,final,"LINESTRING (-1535468.208 -3772745.669, -153547..."
5,coastlines.679,1993,good,final,"LINESTRING (-1535468.984 -3772739.771, -153547..."
6,coastlines.680,1994,good,final,"LINESTRING (-1535469.121 -3772738.723, -153547..."
7,coastlines.681,1995,good,final,"LINESTRING (-1535469.945 -3772732.456, -153547..."


We can now plot the DEA Coastlines annual shoreline data using `geopandas`. 
Zoom in to **Bussleton** in Western Australia on the interactive map to view the shoreline data.

Annual shorelines represent the median or 'most representative' position of the shoreline at approximately mean sea level tide (0 m Above Mean Sea Level) for each year from 1988 onward.
Light-coloured lines (e.g. yellow) in the plot below represent recent shorelines (e.g. 2019), while dark-coloured lines represent older shorelines (e.g. 1988).

> **Note:** For more detail about DEA Coastlines annual shoreline data, refer to the official Geoscience Australia [DEA Coastlines product description](https://cmi.ga.gov.au/data-products/dea/581/dea-coastlines#details).

In [3]:
# Plot on interactive map of DEA Coastlines data within our bounding box
deacl_shorelines_gdf.explore(column='year', cmap='inferno', categorical=True)

### Rates of change statistics points

We can also load and plot the DEA Coastlines rates of change statistics points.
These points provide robust rates of annual coastal change (in metres per year) for every 30 m along Australia's non-rocky (e.g. sandy and muddy) coastlines. 
These rates are calculated by linearly regressing annual shoreline positions against time, using the most recent shoreline as a baseline.

Negative values (red points) indicate retreat (e.g. erosion), and positive values indicate growth (e.g. progradation) over time.
By default, rates of change are shown for points with a statistically significant trend over time only.

> **Note:** For more detail about DEA Coastlines rates of change statistics data, refer to the official Geoscience Australia [DEA Coastlines product description](https://cmi.ga.gov.au/data-products/dea/581/dea-coastlines#details).

In [4]:
# Load DEA Coastlines annual shorelines data within our bounding box
deacl_statistics_gdf = get_coastlines(bbox=(xmin, ymin, xmax, ymax),
                                      layer='statistics')

# Filter points to statistically significant results only
deacl_statistics_gdf = deacl_statistics_gdf.loc[
    deacl_statistics_gdf.sig_time < 0.01]

# Plot on interactive map
deacl_statistics_gdf.explore(column='rate_time',
                             cmap='RdBu',
                             vmin=-3,
                             vmax=3,
                             tooltip='rate_time')

The DEA Coastlines rates of change statistics points data also provides many additional statistics that give insights into coastal change in Australia.
For a full description of each of these statistics, refer to the Rates of change statistics section of the official Geoscience Australia [DEA Coastlines product description](https://cmi.ga.gov.au/data-products/dea/581/dea-coastlines#details).

In [5]:
deacl_statistics_gdf.head()

,id,rate_time,sig_time,se_time,outl_time,rate_soi,sig_soi,se_soi,outl_soi,dist_1988,...,dist_2019,dist_2020,valid_obs,valid_span,sce,nsm,max_year,min_year,abs_time,geometry
60,coastlines_statistics.45321,-5.28,0.0,0.27,None,-2.72,0.837,13.12,None,135.25,...,2.22,0,33,33,140.42,-135.25,1988,2012,5.28,POINT (-1537106.894 -3773090.348)
61,coastlines_statistics.45322,-4.81,0.0,0.25,None,-2.30,0.849,11.98,None,126.63,...,1.44,0,33,33,128.53,-126.63,1988,2012,4.81,POINT (-1537136.462 -3773089.887)
59,coastlines_statistics.45320,-5.55,0.0,0.28,None,-2.77,0.842,13.78,None,138.56,...,1.34,0,33,33,144.33,-138.56,1988,2014,5.55,POINT (-1537077.305 -3773087.017)
62,coastlines_statistics.45323,-4.09,0.0,0.22,None,-1.57,0.879,10.23,None,109.15,...,2.55,0,33,33,110.95,-109.15,1989,2011,4.09,POINT (-1537165.851 -3773085.447)
58,coastlines_statistics.45319,-5.69,0.0,0.29,None,-3.22,0.822,14.15,None,139.46,...,-0.91,0,33,33,143.32,-139.46,1988,2014,5.69,POINT (-1537047.956 -3773083.511)


### Exporting DEA Coastlines data as vector files
We can easily export the loaded data as spatial vector files (e.g. ESRI Shapefiles or GeoJSON) so that they can be analysed further in GIS software:

In [6]:
deacl_shorelines_gdf.to_file('deacoastlines_shorelines.shp')
deacl_statistics_gdf.to_file('deacoastlines_statistics.shp')

## DEA Coastlines analysis tools

The following sections provide useful tools for analysing DEA Coastlines data directly within a Jupyter Notebook without needing to load the data into GIS software.
All outputs from the tools below will be saved to a new folder in this directory called `deacoastlines_outputs`.

### Extract shoreline data by coastal transect
This tool allows you to interactively draw a transect over DEA Coastlines annual shoreline data, and get back a graph showing how shoreline positions have changed over time.
To use the tool:

1. Run the cell below; an interactive application will appear 
2. Zoom in on the map and use the `Draw a polyline` tool on the left to draw a transect so that it crosses through a set of shorelines (see example below). Transects should be drawn in a consistent direction, e.g. starting on land and finishing over water. This ensures that results can be compared between transects.

![Transect example](../Supplementary_data/DEA_Coastlines/deacoastlines_transect.jpg)

3. Press `Finish` when you are happy with the line, then click `Done` in the top right
4. A graph will appear below the map showing distances along the transect to each annual shoreline (distances will be measured from the start of the transect line you drew above)

> **Optional:** Set transaction extraction mode to `'width'` and re-run the cell below to measure the width between two adjacent sets of coastlines (e.g. across the neck of a tombolo or sandbar). 

> **Optional:** Upload a GeoJSON or ESRI Shapefile containing transects using the `Upload` button to automatically extract shoreline positions from multiple coastal transects (uploaded files should be less than 5 megabytes in size).

In [7]:
deacoastlines.transect_app()


transect_app(children=(GridspecLayout(children=(HTML(value='<h3>Digital Earth Australia Coastlines shoreline t…

The resulting figure shows how shorelines at this location have changed over time.

In `distance` mode, the y-axis shows the distance from the start of the transect to the location at which the transect intersects with each annual shoreline (shown on the x-axis).
If transects were drawn with a starting point on land, decreasing distances over time indicate the coastline at this location was retreating or eroding since 1988.
Conversely, increasing distances over time indicate the coastline was growing.

In `width` mode, the y-axis shows the distance (i.e. width) between the first and last time the transect intersects each annual shoreline. Increasing widths indicate that a sandbar, spit or peninsular was growing wider over time; decreasing widths indicate that it was eroding or growing narrower.

## Next steps: coastline erosion notebook

The [Coastal erosion notebook](../Real_world_examples/Coastal_erosion.ipynb) in this repository provides a simplified example of the method used to extract DEA Coastlines annual shoreline data. 
Run this notebook if you would like to generate more customised coastlines for a specific location, time period, epoch (e.g. annual or biennial coastlines) or tidal range (e.g. the position of the coastline at low, mid  or high tide).

> **Note:** This notebook currently supports extracting coastline data only, not generating rates of change statistics like those included in the DEA Coastlines product.

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** January 2022

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)